In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y


In [2]:
X = torch.tensor([[0.,1.,2.], [3.,4.,5.], [6.,7.,8.]])
K = torch.tensor([[0., 1.], [2., 3.]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [3]:
# 实现二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [9]:
# 垂直边缘检测
x = torch.ones((6, 8))
x[:, 2:6] = 0
K = torch.tensor([[1. ,-1.]])
Y = corr2d(x, K)
print(Y)
corr2d(x.t(), K) #只能检测垂直边缘，水平边缘无法检测

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [10]:
# 学习卷积核
# 构造一个二维卷积层，输入和输出通道都为1，形状为(1,2)
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias = False)

# batchsize, channel. height, weight
X = x.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    #迭代卷积核
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch{i+1}, loss {l.sum():.3f}')

batch2, loss 7.111
batch4, loss 2.302
batch6, loss 0.840
batch8, loss 0.327
batch10, loss 0.131


In [11]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0288, -0.9546]])

In [14]:
X = torch.tensor([[ 1.,  0.,  0.,  0.,  0., 0.,  0.],
        [ 0.,  1.,  0.,  0.,  0., 0.,  0.],
        [ 0.,  0.,  1.,  0.,  0., 0.,  0.],
        [ 0.,  0.,  0.,  1.,  0., 0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., 0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., 1.,  0.],
        [ 0.,  0.,  0.,  0.,  0., 0.,  1.]])

K = torch.tensor([[1. ,-1.]])
corr2d(X, K)

tensor([[ 1.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.],
        [ 0.,  0.,  0.,  0.,  0., -1.]])

In [16]:
corr2d(X.t(), K)

tensor([[ 1.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.],
        [ 0.,  0.,  0.,  0.,  0., -1.]])

In [17]:
corr2d(X, K.t())

tensor([[ 1., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1., -1.]])